In [25]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from tqdm.notebook import tqdm

# train_folder = "../../../Datasets/dataset_pickle/test/train/"
# eval_folder = "../../../Datasets/dataset_pickle/test/eval/"

train_folder = "../data/test/train/"
eval_folder = "../data/test/eval/"


epochs = 10
batch_size = 2
hidden_size = 10
output_size = 1  # Binary classification
number_of_eeg_channels = 2
eval_batch = 1

In [26]:
# Custom dataset class
class EEGDataset(Dataset):
    def __init__(self, folder_path):
        self.data = self.load_data(folder_path)

    def load_data(self, folder_path):
        data = []
        labels = []
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".npz"):
                file_path = os.path.join(folder_path, file_name)
                data.append(file_path)
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        file_path = self.data[idx]
        npz_data = np.load(file_path)
        eeg_data = npz_data['array1'][0:number_of_eeg_channels]
        label = npz_data['array2']
        return torch.tensor(eeg_data, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

# def collate(batch):
#     return batch

# Load data from eval and train folders
train_dataset = EEGDataset(train_folder)
val_dataset = EEGDataset(eval_folder)
eval_dataset = EEGDataset(eval_folder)

# Split the data into training and validation sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)#, collate_fn=collate)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
eval_loader = DataLoader(eval_dataset, batch_size=eval_batch, shuffle=False)

In [10]:

# Making a causal conv1d to later use in the conv_TCN

class CausalConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation=1):
        super(CausalConv1d, self).__init__()

        # Pad the input to ensure causality
        padding = (kernel_size - 1) * dilation

        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, padding=padding, dilation=dilation)

    def forward(self, x):
        return self.conv(x)

In [27]:
# Creating a TCN Block using causal networks

class TcnEEGModel(nn.Module):
    def __init__(self, in_channels, input_features, output_size):
        super(TcnEEGModel, self).__init__()
        self.conv1 = nn.Sequential(
            nn.utils.weight_norm(nn.Conv1d(in_channels, 32, 3, padding = 2, dilation=1), dim = None),
            nn.ReLU(),
            nn.Dropout(0.2),
        )
        self.conv2 = nn.Sequential(
            nn.utils.weight_norm(nn.Conv1d(in_channels, 32, 3, padding = 4, dilation=2), dim = None),
            nn.ReLU(),
            nn.Dropout(0.2),
        )
        self.residual1 = nn.Conv1d(in_channels, 64, 1)
        self.conv3 = nn.Sequential(
            nn.utils.weight_norm(nn.Conv1d(in_channels, 32, 3, padding = 8, dilation=4), dim = None),
            nn.ReLU(),
            nn.Dropout(0.2),
        )
        self.conv4 = nn.Sequential(
            nn.utils.weight_norm(nn.Conv1d(in_channels, 32, 3, padding = 16, dilation=8), dim = None),
            nn.ReLU(),
            nn.Dropout(0.2),
        )
        self.residual2 = nn.Conv1d(64, 32, 1)
        self.linear = nn.Linear(64, 1)

    def forward(self, src):

        residual = src
        
        src = self.conv1(src)
        src = self.conv2(src)
        src += self.residual1(residual)

        residual = src


        src = self.conv3(src)
        src = self.conv4(src)
        src += self.residual2(residual)

        src = src.squeeze(1)
        src = src.permute(0,2,1)
        src = torch.mean(src, dim=1)
        src = self.linear(src)
        src = torch.sigmoid(src)
        return src
        
            

In [29]:
input_channels = train_dataset.__getitem__(0)[0].shape[0]
input_features = train_dataset.__getitem__(0)[0].shape[1]
criterion = nn.CrossEntropyLoss()
loss_fn = torch.nn.BCELoss()
# optimizer = torch.optim.AdamW(params, lr=0.0003)
# optimizer = torch.optim.Adam(params, lr=0.0003, betas=(0.9, 0.98), eps=1e-9)

model = TcnEEGModel(input_channels, input_features, output_size)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0003)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/home/arsalan/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


TcnEEGModel(
  (conv1): Sequential(
    (0): Conv1d(2, 32, kernel_size=(3,), stride=(1,), padding=(2,))
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
  )
  (conv2): Sequential(
    (0): Conv1d(2, 32, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(2,))
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
  )
  (residual1): Conv1d(2, 64, kernel_size=(1,), stride=(1,))
  (conv3): Sequential(
    (0): Conv1d(2, 32, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(4,))
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
  )
  (conv4): Sequential(
    (0): Conv1d(2, 32, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(8,))
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
  )
  (residual2): Conv1d(64, 32, kernel_size=(1,), stride=(1,))
  (linear): Linear(in_features=64, out_features=1, bias=True)
)

In [30]:
num_training_steps = epochs * len(train_loader)
# num_eval_steps = num_epochs * len(eval_dataloader)

print(num_training_steps)


170


In [31]:
def eval(model, data_loader, loss_fn,eval_batch):
  model.eval()
  print(len(data_loader))
  eval_loss = 0.0
  correct_eval = 0.0
  # print(data_loader.shape)
  with torch.no_grad():
    for x, y in tqdm(data_loader):
      file = x.to(device)
      label = y.to(device)
      output = model(file)
      if(eval_batch == 1):
        loss = loss_fn(output.squeeze(0), label)
      else:
        loss = loss_fn(output.squeeze(0), label.view(-1, 1))
      eval_loss += loss.item()

      preds = torch.where(output > 0.5, 1, 0).T
      correct_eval += (preds == label).sum().item()
      # print(correct_eval)

    eval_loss = eval_loss/len(data_loader)
    eval_acc = (correct_eval/(len(data_loader)*batch_size))

  return eval_loss, eval_acc

prev_eval_loss, prev_eval_acc = eval(model, val_loader, loss_fn, 1)
print("Loss:",prev_eval_loss, "Accuracy",prev_eval_acc)

4


  0%|          | 0/4 [00:00<?, ?it/s]

RuntimeError: Given groups=1, weight of size [32, 2, 3], expected input[1, 32, 120002] to have 2 channels, but got 32 channels instead

In [ ]:
# Create a simple 1D convolutional neural network model
class ConvEEGModel(nn.Module):
    def __init__(self, input_channels, input_features, output_size):
            super(ConvEEGModel, self).__init__()
            self.conv1 = nn.Sequential(
                nn.Conv1d(input_channels, 32, 3, padding=1),
                nn.BatchNorm1d(32),
                nn.GELU(),
                # nn.MaxPool1d(2,2)        #16*16*16
            )

            self.conv2 = nn.Sequential(
                nn.Conv1d(32, 64, 3, padding=1),
                nn.BatchNorm1d(64),
                nn.GELU(),
                # nn.MaxPool2d((2,2), (2,2))        #32*16*16
            )
            self.conv3 = nn.Sequential(
                nn.Conv2d(64, 96, 3, padding=1),
                nn.BatchNorm2d(96),
                nn.GELU(),
                nn.Conv2d(96, 128, 3, padding=1),
                nn.BatchNorm2d(128),
                nn.GELU(),
                # nn.MaxPool2d((2, 1), (2, 1)),  #64*8*8
                nn.Dropout2d(0.2),
            )
            self.conv4 = nn.Sequential(
                nn.Conv2d(128, 192, 3, padding=1),
                nn.BatchNorm2d(192),
                nn.GELU(),
                nn.Conv2d(192, 256, 3, padding=1),
                nn.BatchNorm2d(256),
                nn.GELU(),
                # nn.MaxPool2d((2, 1), (2, 1)),  # 64*4*4  
                nn.Dropout2d(0.2),
            )
            self.conv5 = nn.Sequential(
                # nn.Conv2d(256, 1, 4),
                nn.Conv2d(256, 1, 3, padding='same'),
                nn.BatchNorm2d(1),
                nn.GELU(),
            )  

            self.linear = nn.Linear(64, 1)

        # self.conv1d = nn.Conv1d(in_channels=input_channels, out_channels=64, kernel_size=3)
        # self.relu = nn.ReLU()
        # self.flatten = nn.Flatten()
        # self.fc1 = nn.Linear(64 * (input_features - 2), output_size)
        # self.softmax = nn.Softmax(dim=1)

    def forward(self, src):
            src = self.conv1(src)
            # print(src.shape)                                 # (*, 16, 32, 256)
            src = self.conv2(src)
            # print(src.shape)                                 # (*, 32, 16, 128)
            # src = self.conv3(src)
            # # print(x.shape)                                 # (*, 64, 8, 128)
            # src = self.conv4(src)
            # # print(x.shape)                                 # (*, 128, 4, 128)        
            # src = self.conv5(src)
            # print(x.shape)                                 # (*, 256, 1, 125)
            src = src.squeeze(1)
            # print("squeez", src.shape)
            src = src.permute(0,2,1)
            # print("permute", src.shape)
            # src = self.encoder(src)
            src = torch.mean(src, dim=1)
            # print("mean", src.shape)
            src = self.linear(src)
            src = torch.sigmoid(src)
            # print("sigmoid", src.shape)
            # src = self.fc1(src)
            # # print(x.shape)
            # src = self.softmax(src)
            return src

        # # print(x.shape)
        # x = self.conv1d(x)
        # # print(x.shape)
        # x = self.relu(x)
        # # print(x.shape)
        # x = self.flatten(x)
        # # print(x.shape)
        # x = self.fc1(x)
        # # print(x.shape)
        # x = self.softmax(x)
        # # print(x.shape)
        # return x

input_channels = train_dataset.__getitem__(0)[0].shape[0]
input_features = train_dataset.__getitem__(0)[0].shape[1]
criterion = nn.CrossEntropyLoss()
loss_fn = torch.nn.BCELoss()
# optimizer = torch.optim.AdamW(params, lr=0.0003)
# optimizer = torch.optim.Adam(params, lr=0.0003, betas=(0.9, 0.98), eps=1e-9)

model = ConvEEGModel(input_channels, input_features, output_size)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0003)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
a